In [31]:
import numpy as np 
import ultranest

from math import factorial

In [32]:
# read phases from file
tess_phases = np.loadtxt("../data/tess_phases.txt")
cheops_phases = np.loadtxt("../data/cheops_phases.txt")

# weigh by observing cadence
weights = np.concatenate([np.ones_like(cheops_phases) * 10. / 60. / 60. / 24., np.ones_like(tess_phases) * 2. / 60. / 24.] )
obs_phases = np.concatenate([cheops_phases, tess_phases])

# flare phases
phases = np.array([0.61248919, 0.81165721, 0.01788908, 0.0296636,  0.05760315, 0.04067287,
0.73005547, 0.94878914, 0.11323833, 0.20031473, 0.15087211, 0.04514247,
0.02527212, 0.05657772, 0.06247738, ]) 

# phases = np.random.rand(15)

# shift by 0.5
obs_phases = (obs_phases + 0.5) % 1
phases = (phases + 0.5) % 1

# define binning
nbins = 75
bins = np.linspace(0, 1, nbins)
binmids= (bins[1:] + bins[:-1]) / 2

# bin the phases
arr = np.digitize(obs_phases, bins)

# sum the observing times in each bin to binned weights
# unit of entries in binned is [days]
binned = np.array([np.sum(weights[arr==i]) for i in range(1, len(bins))]) 

print(len(phases) / np.sum(binned))


# define the two models we want to compare
def modulated_model(binmids, lambda0, lambda1, phase0, dphase, weight=binned):
    mask = (binmids > phase0) & (binmids < (phase0 + dphase)%1)
    result = np.zeros_like(binmids)

    # multiply by weight because that modifies the measured rate
    result[~mask] = lambda0 * weight[~mask]
    result[mask] = lambda1 * weight[mask]

    return result #number of observed flares per bin

def unmodulated_model(lambda0, weight=binned):
    return lambda0 * weight #number of observed flares per bin

# observed:
hist, bins = np.histogram(phases, bins=bins)

# define the factorials for the numbers in hist for the likelihood computation
factorials = np.array([factorial(h) for h in hist])

# Poisson log-likelihood function
def likelihood_poisson(rate, hist, factorials):
    logs = -rate - np.log(factorials) + np.log(rate) * hist
    return np.sum(logs)

# log-likelihood for the modulated model
def likelihood_mod(params):

    rate = modulated_model(binmids, *params, weight=binned)

    return likelihood_poisson(rate, hist, factorials)

p1 = ["lambda0", "lambda1", "phase0", "dphase"]

def prior_transform_mod(cube):
    # the argument, cube, consists of values from 0 to 1
    # we have to convert them to physical scales

    params = cube.copy()
    # lambda0 from 0 to 4 flat
    params[0] = cube[0] * 4 
    # lambda1 from 0 to 4 flat
    params[1] = cube[1] * 4
    # phase zero from 0 to 1
    params[2] = cube[2]
    # dphase from 0 to 0.5
    params[3] = cube[3] * 0.5
    return params


p0 = ["lambda0"]

def prior_transform_unmod(cube):
    # the argument, cube, consists of values from 0 to 1
    # we have to convert them to physical scales

    params = cube.copy()
    # lambda0 from 0 to 4 flat
    params[0] = cube[0] * 4
  
    return params



# define log-likelihood, prior, and probability
def likelihood_unmod(params):
    lambda0 = params[0]
    rate = unmodulated_model(lambda0, weight=binned)
    return likelihood_poisson(rate, hist, factorials)



0.20489794627752925


In [33]:

p1 = ["lambda0", "lambda1", "phase0", "dphase"]

sampler1 = ultranest.ReactiveNestedSampler(p1, likelihood_mod, prior_transform_mod)

sampler0 = ultranest.ReactiveNestedSampler(p0, likelihood_unmod, prior_transform_unmod)

In [34]:
result1 = sampler1.run(min_num_live_points=7000)
sampler1.print_results()

[ultranest] Sampling 7000 live points from prior ...


[ultranest] Explored until L=-3e+01   [-28.9527..-28.9527]*| it/evals=110040/653768 eff=17.0138% N=7000 
[ultranest] Likelihood function evaluations: 653781
[ultranest]   logZ = -40.04 +- 0.02276
[ultranest] Effective samples strategy satisfied (ESS = 59643.1, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.46+-0.02 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.02, need <0.5)
[ultranest]   logZ error budget: single: 0.03 bs:0.02 tail:0.01 total:0.02 required:<0.50
[ultranest] done iterating.

logZ = -40.037 +- 0.048
  single instance: logZ = -40.037 +- 0.032
  bootstrapped   : logZ = -40.038 +- 0.047
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    lambda0             : 0.000 │▁▁▂▄▆▇▇▇▇▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ │0.543     0.118 +- 0.057
    lambda1             : 0.00  │▁▁▂▅▇▇▇▆▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁│4.00      0.89 +- 0.58
    phase0              : 0.00 

In [35]:
result0 = sampler0.run(min_num_live_points=400)
sampler0.print_results()

[ultranest] Sampling 400 live points from prior ...


[ultranest] Explored until L=-4e+01   [-38.2277..-38.2277]*| it/evals=2640/3108 eff=97.4889% N=400  
[ultranest] Likelihood function evaluations: 3148
[ultranest]   logZ = -41.61 +- 0.0576
[ultranest] Effective samples strategy satisfied (ESS = 1271.9, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.46+-0.07 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.07, need <0.5)
[ultranest]   logZ error budget: single: 0.09 bs:0.06 tail:0.04 total:0.07 required:<0.50
[ultranest] done iterating.

logZ = -41.625 +- 0.127
  single instance: logZ = -41.625 +- 0.085
  bootstrapped   : logZ = -41.608 +- 0.122
  tail           : logZ = +- 0.036
insert order U test : converged: True correlation: inf iterations

    lambda0             : 0.047 │ ▁ ▁▁▁▂▃▃▄▅▆▆▇▇▇▇▇▆▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁ │0.467     0.219 +- 0.055



In [36]:
K = np.exp(result1['logz'] - result0['logz'])
print("K = %.2f" % K)
print("The modulated model is %.2f times more probable than the no-signal model" % K)
print("assuming the models are equally probable a priori.")



K = 4.90
The modulated model is 4.90 times more probable than the no-signal model
assuming the models are equally probable a priori.


50 3.34 3.8

150 5.74 6.5

100 5.26 5.6

75  4.9 4.5

In [ ]:
import pandas as pd

In [46]:
tess_phases = np.loadtxt("../data/tess_phases.txt")
cheops_phases = np.loadtxt("../data/cheops_phases.txt")
flares = pd.read_csv("../results/flare_phases_and_energies.csv")
flares = flares.sort_values("ed_rec", ascending=True).iloc[1:] # exclude the smallest flare

# weigh by observing cadence
weights = np.concatenate([np.ones_like(cheops_phases) * 10. / 60. / 60. / 24., np.ones_like(tess_phases) * 2. / 60. / 24.] )
obs_phases = np.concatenate([cheops_phases, tess_phases])

In [49]:
mask = obs_phases < 0.2
nsmall, nbig = weights[mask].sum(), weights[~mask].sum()
nsmall, nbig, nsmall + nbig

(17.949305555555554, 55.257870370370355, 73.20717592592591)